In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('x.csv', sep=',',header=None)
df = df.values

In [2]:
df.shape

(10000, 3)

In [3]:
x, y, z = df[:,0], df[:,1], df[:,2]


In [4]:
#Draw x to see the suitable number of classes to describe x
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

ax = plt.subplot(111, projection='3d')  #draw 3d graphs
ax.scatter(x, y, z)
#three axises
ax.set_zlabel('Z')
ax.set_ylabel('Y')
ax.set_xlabel('X')
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
def multiPDF(x, mu, sigma):
    """
    Probability density function of multivariate normal distribution
    """
    size = len(x)
    if size ==len(mu) and (size, size) == sigma.shape:
        detSigma = np.linalg.det(sigma)
        try :
            div = 1.0 / (np.math.pow(2 * np.pi, len(x) / 2.0) * np.math.pow(detSigma, 1.0 / 2))
        except:
            print ("except",detSigma)
        x_mu = np.matrix(x - mu)
        invSigma = np.linalg.inv(sigma)
        numerator = np.math.pow(np.math.e, -0.5 * (x_mu.T) * invSigma * x_mu)
        result = numerator * div
        return result
    else:
        return -1

def logLikelihood(dataSet, pi, Mu, Sigma):
    K = len(pi)
    N, M = np.shape(dataSet)
    P = np.zeros([N, K])
    for k in range(K):
        for i in range(N):
            P[i, k] = multiPDF(dataSet[i, :][None].T, Mu[:, k][None].T, Sigma[:, :, k])
    result = np.sum(np.log(np.dot(P,pi)))
    return result
def initEM(dataSet, K):
    """
    N: sample size (10000)
    M: sample dimension (d=3)
    K: number of classes
    
    calculate posterior probability of zn
    gamma[i,j] represents the probability that i is generated by z
    initialize all parameters
    """
    N, M = dataSet.shape
    gamma = np.zeros([N, K])
    numPerK = N / K
    for k in range(K):
        start = int(np.floor(k * numPerK))
        end = int(np.floor((k + 1) * numPerK))
        gamma[start: end, k] = 1
    pi, Mu, Sigma = Mstep(dataSet, gamma)
    return gamma, pi, Mu, Sigma
def Mstep(dataSet, gamma):
    N, M = dataSet.shape
    K = gamma.shape[1]
    # update pi
    N_k = np.sum(gamma, 0) # summation of sample probability for each component
    pi = N_k / np.sum(N_k)    # Sk[1]/S.[1]
    # update mu (M * K)
    Mu = np.dot(np.dot(dataSet.T,gamma),np.diag(np.reciprocal(N_k)))
    # update covaraince matrix（M*M*K ）
    Sigma = np.zeros([M, M, K])
    for k in range(K):
        dataMeanSub = dataSet.T - np.dot(Mu[:,k][None].T,np.ones([1,N]))
        Sigma[:, :, k] = (dataMeanSub.dot(np.diag(gamma[:, k])).dot(dataMeanSub.T)) / N_k[k]
    return pi, Mu, Sigma

def Estep(dataSet, pi, Mu, Sigma):
    """
    calculate P(z_k|x)
    gamma[i,j] represents the generative probability of i from z_j
    """
    N = dataSet.shape[0]
    K = len(pi)
    # gamma is n*k array
    gamma = np.zeros([N,K])
    for k in range(K):
        for i in range(N):
            gamma[i, k] = pi[k] * multiPDF(dataSet[i, :][None].T, Mu[:, k][None].T, Sigma[:, :, k])
    # To make the summation of each row 1
    gamma = gamma * np.reciprocal(np.sum(gamma, 1)[None].T)
    return gamma

def EM(data,K):
    """
    Use EM to esitmate parameters of GMM
    """
    gamma, pi, Mu, Sigma = initEM(data, K)
    iter = 0
    prevLL = -999999
    valLogLikelihood = []
    while (True):
        gamma = Estep(data, pi, Mu, Sigma)
        pi, Mu, Sigma = Mstep(data, gamma)
        valLL = logLikelihood(data, pi, Mu, Sigma)
        iter = iter + 1
        if iter % 1 == 0 :
            valLogLikelihood.append(valLL)
        if (iter > 200 or abs(valLL - prevLL) < 0.000000000001):
            break
        print(valLL)
        prevLL = valLL
    return pi,Mu,Sigma, gamma

In [6]:
#from the result above, we know that there are three main components
pi, Mu, Sigma, gamma = EM(df,3)

-68789.00060098796
-68788.67796584927
-68788.18821682599
-68787.4078938652
-68786.13968880248
-68784.03842588385
-68780.4803717346
-68774.29028188804
-68763.12492555351
-68741.93806493055
-68698.67382667398
-68600.56035701357
-68349.13583803688
-67726.01743179397
-67010.7160223274
-66773.6284665682
-66694.24925239703
-66650.03004615486
-66615.50233386309
-66584.80160945094
-66555.15623823847
-66523.86288405876
-66487.03051675821
-66437.56909568147
-66361.0033568461
-66230.47834972358
-66027.5369413196
-65836.0907287246
-65756.59355493073
-65733.89903383206
-65724.1883835434
-65717.44477987535
-65711.97043258788
-65707.43786756122
-65703.70151030722
-65700.63385045629
-65698.11492287501
-65696.03833058778
-65694.31452417331
-65692.87057099976
-65691.64805234631
-65690.60037275493
-65689.69014851197
-65688.88691946285
-65688.16520218903
-65687.50279390338
-65686.87917649641
-65686.27380853136
-65685.66398164169
-65685.02168255206
-65684.30838480848
-65683.4655086161
-65682.39540317887
-6

In [7]:
gamma

array([[4.50964840e-14, 2.10929341e-12, 1.00000000e+00],
       [1.73377591e-11, 4.54907809e-19, 1.00000000e+00],
       [5.84642645e-31, 1.00000000e+00, 5.86229055e-22],
       ...,
       [1.00447930e-28, 2.23095164e-15, 1.00000000e+00],
       [8.05973838e-13, 1.00000000e+00, 7.07249611e-14],
       [1.20366305e-18, 1.00000000e+00, 1.51846405e-18]])

In [11]:
pi

array([0.10112533, 0.5970866 , 0.30178807])

In [12]:
Mu

array([[ 5.06429164,  0.26999146, -3.05824344],
       [-0.05344663, -1.32470484,  1.99795608],
       [-4.96956173,  3.67938418, -3.01901106]])

In [10]:
Sigma

array([[[ 1.00242741e+00,  1.58789038e+01,  2.00673664e+00],
        [ 4.40355070e-02, -3.54431203e+00, -4.51688405e-02],
        [-1.63150038e-02, -3.46210958e+00,  5.07228230e-02]],

       [[ 4.40355070e-02, -3.54431203e+00, -4.51688405e-02],
        [ 5.21470977e-01,  1.37650793e+00,  4.85600863e-01],
        [-1.03002703e-02,  8.75301912e-01, -5.21546586e-03]],

       [[-1.63150038e-02, -3.46210958e+00,  5.07228230e-02],
        [-1.03002703e-02,  8.75301912e-01, -5.21546586e-03],
        [ 9.43619683e-01,  2.51556950e+00,  1.94307726e+00]]])

In [13]:
#output gamma
np.savetxt("z.csv", gamma, delimiter=",")